In [7]:
import logging
import os

import pypsa
import yaml
import pandas as pd
import geopandas as gpd
#import geoviews as gv
#import hvplot.pandas
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib as plt

from scipy.sparse import csgraph
from itertools import product

from shapely.geometry import Point, LineString
import shapely, shapely.prepared, shapely.wkt

logger = logging.getLogger(__name__)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 70)

import sys
sys.path.append('../')  # to import helpers
from scripts._helpers import _sets_path_to_root
_sets_path_to_root("pypsa-africa")

In [8]:
import pypsa
import geopandas as gpd
import sys
sys.path.append('../')  # to import helpers
from scripts._helpers import _sets_path_to_root
_sets_path_to_root("pypsa-africa")


In [9]:
network_path = os.getcwd() + "/networks/base.nc"
n = pypsa.Network(network_path)
n

In [12]:
n.bounds

In [18]:
n.buses

In [25]:
regions = os.getcwd() + "/resources/country_shapes.geojson"
gpd.read_file(regions)

In [26]:
regions = os.getcwd() + "/resources/offshore_shapes.geojson"
gpd.read_file(regions)


In [20]:
n.buses["country"].unique

In [12]:
n.lines

In [13]:
# Try to make the points 'right'
# TODO: issue at n.plot in plot.py of PyPSA

# df = n.buses
# df["geometry"] = gpd.points_from_xy(df.lon, df.lat)
# gdf = gpd.GeoDataFrame(df, crs='epsg:4326')
# n.buses = gdf


# #import geoplot
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs

# #Germany shape information
# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# africa = world.query('continent == "Africa"')

# #Boundary coordinates for Germany
# xlim = ([africa.total_bounds[0],  africa.total_bounds[2]])
# ylim = ([africa.total_bounds[1],  africa.total_bounds[3]])

# #plot
# fig,ax = plt.subplots(1,1,subplot_kw={"projection":ccrs.PlateCarree()})
# fig.set_size_inches(15,15)
# ax = n.plot(ax=ax, boundaries = xlim+ylim, bus_sizes=1, geometry = True)

In [14]:
from shapely import wkt

lines = n.lines
lines['geometry'] = lines['geometry'].apply(wkt.loads)
lines = gpd.GeoDataFrame(lines, crs='epsg:4326')

buses = n.buses
buses["geometry"] = gpd.points_from_xy(buses.lon, buses.lat)
buses = gpd.GeoDataFrame(buses, crs='epsg:4326')

In [ ]:
lines = lines[lines["country"]=="nigeria"]
buses = buses[buses["country"]=="nigeria"]

buses.hvplot(
    geo=True,
    size = 10,  # buses["tag_area"]**(0.5)/10,
    frame_height=750,
    alpha=0.4,
    tiles='CartoLight',
    hover_cols=['bus_id'],
    color = 'orange'  
) * lines.hvplot(
    geo=True,
    alpha=0.4,
    hover_cols=['line_id']
).opts(
    active_tools=['pan', 'wheel_zoom']
)

# TEST AREA - to test single functions

In [ ]:
lines = (
        pd.read_csv(
            "data/osm/africa_all_lines_build_network.csv",
            dtype=dict(
                line_id="str",
                bus0="str",
                bus1="str",
                underground="bool",
                under_construction="bool",
            ),
        )
        .set_index("line_id")
        .rename(columns=dict(voltage="v_nom", circuits="num_parallel"))
    )

lines["length"] /= 1e3  # m to km conversion
lines["v_nom"] /= 1e3  # V to kV conversion
lines = lines.loc[:, ~lines.columns.str.contains("^Unnamed")]  
lines = _rebase_voltage_to_config(lines)

#lines

In [ ]:
def _rebase_voltage_to_config(component):
    """
    Rebase the voltage of components to the config.yaml input

    Components such as line and buses have voltage levels between
    110 kV up to around 850 kV. PyPSA-Africa uses 3 voltages as config input.
    This function rebases all inputs to the lower, middle and upper voltage
    bound.

    Parameters
    ----------
    component : dataframe
    """
    v_min = 110  # min. filtered value in dataset
    v_low = 110
    v_mid = 220
    v_up = 330
    v_low_mid = (v_mid - v_low) / 2 + v_low  # between low and mid voltage
    v_mid_up = (v_up - v_mid) / 2 + v_mid  # between mid and upper voltage
    component.loc[
        (v_min <= component["v_nom"]) & (component["v_nom"] < v_low_mid), "v_nom"
    ] = v_low
    component.loc[
        (v_low_mid <= component["v_nom"]) & (component["v_nom"] < v_mid_up), "v_nom"
    ] = v_mid
    component.loc[v_mid_up <= component["v_nom"], "v_nom"] = v_up

    return component
